In [8]:
import pprint
from functools import partial

import pandas as pd
import numpy as np

from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.trainer import Trainer

In [9]:
import mxnet as mx
print(mx.__version__)

import gluonts
print(gluonts.__version__)

1.4.1
0.4.0


In [10]:
seed=42
epochs = 300
num_batches_per_epoch = 400
ctx="cpu"

datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch,
            ctx=ctx
        ),
    ),
]

def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict

if __name__ == "__main__":
    
    mx.random.seed(seed)
    np.random.seed(seed)

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

df[["dataset", "estimator", "MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[24000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="3M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=8, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=mxnet.context.Context("cpu", 0), epochs=300, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=400, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='3M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='24000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=8>, train=<gluonts.dataset.common.File

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 400/400 [01:19<00:00,  5.02it/s, avg_epoch_loss=7.44]
INFO:root:Epoch[0] Elapsed time 79.782 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.435294
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 400/400 [00:54<00:00,  7.39it/s, avg_epoch_loss=7.16]
INFO:root:Epoch[1] Elapsed time 54.125 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.159251
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 400/400 [01:09<00:00,  5.73it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[2] Elapsed time 69.821 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.939147
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 400/400 [00:40<00:00,  9.92it/s, avg_epoch_loss=6.9] 
INFO:root:Epoch[3] Elapsed time 40.303 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.899338
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 400/400 [00:42<00:00,  9.38it/s, a

100%|██████████| 400/400 [00:43<00:00,  9.22it/s, avg_epoch_loss=6.27]
INFO:root:Epoch[36] Elapsed time 43.378 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.269771
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 400/400 [00:42<00:00,  9.51it/s, avg_epoch_loss=6.36]
INFO:root:Epoch[37] Elapsed time 42.086 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.363293
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 400/400 [00:51<00:00,  7.75it/s, avg_epoch_loss=6.32]
INFO:root:Epoch[38] Elapsed time 51.603 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.318081
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 400/400 [01:20<00:00,  5.00it/s, avg_epoch_loss=6.28]
INFO:root:Epoch[39] Elapsed time 80.062 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.284593
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 400/400 [01:36<00:00,  4.17it/s, avg_epoch_loss=6.34]
INFO:root:Epoch[40] Elapsed time 96.045 s

INFO:root:Epoch[73] Learning rate is 0.001
100%|██████████| 400/400 [01:06<00:00,  5.98it/s, avg_epoch_loss=6.15]
INFO:root:Epoch[73] Elapsed time 66.885 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=6.146993
INFO:root:Epoch[74] Learning rate is 0.001
100%|██████████| 400/400 [00:51<00:00,  7.72it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[74] Elapsed time 51.848 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.243522
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 400/400 [00:52<00:00,  7.60it/s, avg_epoch_loss=6.19]
INFO:root:Epoch[75] Elapsed time 52.623 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.189233
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 400/400 [00:46<00:00,  8.57it/s, avg_epoch_loss=6.25]
INFO:root:Epoch[76] Elapsed time 46.695 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.251016
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 400/400 [00:37<00:00, 10.72it/s, avg_epoch_loss=6.13

...

{'Coverage[0.1]': 0.13391666666666666,
 'Coverage[0.2]': 0.24929166666666666,
 'Coverage[0.3]': 0.36009375,
 'Coverage[0.4]': 0.468921875,
 'Coverage[0.5]': 0.5714635416666667,
 'Coverage[0.6]': 0.6576041666666667,
 'Coverage[0.7]': 0.7480833333333333,
 'Coverage[0.8]': 0.8325677083333334,
 'Coverage[0.9]': 0.9116354166666667,
 'MAE_Coverage': 0.04817534722222223,
 'MASE': 1.1437503232055601,
 'MSE': 1731338.197216394,
 'MSIS': 12.041492755444388,
 'ND': 0.09236888879398086,
 'NRMSE': 0.22024229362086406,
 'QuantileLoss[0.1]': 47662655.25577393,
 'QuantileLoss[0.2]': 73830038.16927797,
 'QuantileLoss[0.3]': 91054187.21316221,
 'QuantileLoss[0.4]': 101445117.35094604,
 'QuantileLoss[0.5]': 105953957.24710846,
 'QuantileLoss[0.6]': 104504613.38739014,
 'QuantileLoss[0.7]': 97184405.04246217,
 'QuantileLoss[0.8]': 82601267.0003601,
 'QuantileLoss[0.9]': 57606974.24379882,
 'RMSE': 1315.803251712198,
 'abs_error': 105953957.29371643,
 'abs_target_mean': 5974.344119287491,
 'abs_target_sum'

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_yearly.


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[23000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="12M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=6, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=mxnet.context.Context("cpu", 0), epochs=300, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=400, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='12M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='23000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=6>, train=<gluonts.dataset.common.Fi

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 400/400 [00:25<00:00, 15.50it/s, avg_epoch_loss=7.29]
INFO:root:Epoch[0] Elapsed time 25.815 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.286303
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 400/400 [00:27<00:00, 14.63it/s, avg_epoch_loss=7.06]
INFO:root:Epoch[1] Elapsed time 27.345 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.055185
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 400/400 [00:25<00:00, 15.83it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[2] Elapsed time 25.290 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.616053
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 400/400 [00:35<00:00, 11.28it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[3] Elapsed time 35.480 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.607514
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 400/400 [00:25<00:00, 15.46it/s, a

KeyboardInterrupt: 

In [ ]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]]

In [ ]:
!jupyter nbconvert --output-dir="./html_outputs" --to html Test_notebook_for_gpu.ipynb

In [ ]:
print("Done!")